In [1]:
from redisvl.extensions.router import Route


# Define routes for the semantic router
technology = Route(
    name="technology",
    references=[
        "what are the latest advancements in AI?",
        "tell me about the newest gadgets",
        "what's trending in tech?"
    ],
    metadata={"category": "tech", "priority": 1},
    distance_threshold=1.0
)

sports = Route(
    name="sports",
    references=[
        "who won the game last night?",
        "tell me about the upcoming sports events",
        "what's the latest in the world of sports?",
        "sports",
        "basketball and football"
    ],
    metadata={"category": "sports", "priority": 2},
    distance_threshold=0.5
)

entertainment = Route(
    name="entertainment",
    references=[
        "what are the top movies right now?",
        "who won the best actor award?",
        "what's new in the entertainment industry?"
    ],
    metadata={"category": "entertainment", "priority": 3},
    distance_threshold=0.7
)

In [2]:
import os
from redisvl.extensions.router import SemanticRouter
from redisvl.utils.vectorize import HFTextVectorizer

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Initialize the SemanticRouter
router = SemanticRouter(
    name="topic-router",
    vectorizer=HFTextVectorizer(),
    routes=[technology, sports, entertainment],
    redis_url="redis://localhost:6379",
    overwrite=True # Blow away any other routing index with this name
)

In [8]:
res = router("tell me about AI")


 match=[b'route_name', b'technology', b'distance', b'0.661981523037']


In [9]:
res

RouteMatch(name='technology', distance=0.661981523037)

In [10]:
res.name

'technology'

In [11]:
res.distance

0.661981523037

In [23]:
from redisvl.extensions.threshold_optimizer.schema import TestData

test_dict = [
    {
        "query": "tell me about tech",
        "query_match": "technology",
    },
    {
        "query": "sports are my favorite",
        "query_match": "sports",
    },
]

test_data = [TestData(**data) for data in test_dict]

In [24]:
from ranx import Run, evaluate

def generate_run_router(test_data, router):
    run_dict = {}

    for td in test_data:
        run_dict[td.q_id] = {}
        route_match = router(td.query)
        print(f"{td.query=}\n\n{route_match=}\n\n")
        if route_match and route_match.name == td.query_match:
            run_dict[td.q_id][td.query_match] = 1
        else:
            run_dict[td.q_id]["no_match"] = 1
            
    return Run(run_dict)

In [25]:
from redisvl.extensions.threshold_optimizer import metrics

qrels = metrics.format_qrels(test_data)
run = generate_run_router(test_data, router)
f1 = evaluate(qrels, run, "f1", make_comparable=True)



 match=[b'route_name', b'technology', b'distance', b'0.566104531288']
td.query='tell me about tech'

route_match=RouteMatch(name='technology', distance=0.566104531288)


td.query='sports are my favorite'

route_match=RouteMatch(name=None, distance=None)




In [26]:
f1

0.5